# Animating eReefs Model Results

This notebook demonstrates how to create some fun animations from eReefs model results.

If you have not already perused them, good precursors to this example are the basic plotting example [plot.ipynb](./plot.ipynb). Please consider running that example first.

---

## Side quest: animating eReefs model results in the CSIRO eReefs Data Explorer

The instructions in the [data-discovery](./data-discovery.ipynb) notebook explain how to use the data explorer to identify interesting eReefs model results variables. 

From where we left our map in those instructions, we have an ammonia layer added, and visualised:

<img src="_static/003_plotted-1.png" alt="screenshot of the ammonia layer from the GBR4_H2p0_B2p0_Chyd_Dcrt dataset in the data explorer" style="align: left; max-width: 800px"/>

From this, you can generate an animation from within the web application by clicking the movie camera icon in the top right corner on our layer box. This should bring up an animation screen as shown below:

<img src="_static/005_animation_ammonia.png" alt="screenshot of an animation for the ammonia layer from the GBR4_H2p0_B2p0_Chyd_Dcrt dataset in the data explorer" style="align: left; max-width: 800px"/>

## Animations in Python

If the animations possible in the data explorer aren't quite what you're looking for, though, it is completely possible to use [emsarray](https://emsarray.readthedocs.io/en/stable/) to create complex animations from your python code.

The examples below use an OPeNDAP endpoint of <https://dapds00.nci.org.au/thredds/dodsC/fx3/gbr4_bgc_GBR4_H2p0_B3p1_Cfur_Dnrt.ncml> which belongs to the [GBR4 Biogeochemistry and Sediments v3.1 near-real-time model results](https://marlin.csiro.au/geonetwork/srv/eng/catalog.search#/metadata/7bc0d1fc-d0e4-4c1a-956f-8f7cd1a6892d) dataset aggregation.

In [ ]:
# First, prepare the environment and access the dataset

import datetime
import emsarray
import shapefile

from matplotlib.figure import Figure
from IPython.display import HTML

%matplotlib notebook
shapefile.VERBOSE = False

dataset = emsarray.open_dataset('https://dapds00.nci.org.au/thredds/dodsC/fx3/gbr4_bgc_GBR4_H2p0_B3p1_Cfur_Dnrt.ncml')

### Animate a horizontal layer over time

Use emsarray to animate how the Total Chloropyll (Chl_a_sum) variable from the model results changes over time for a single depth.

In [ ]:
from emsarray.utils import datetime_from_np_time

def make_title(np_time):
    """Generate a title for a frame of animation."""
    long_name = "Total Chlorophyll at k=-3"
    py_time = datetime_from_np_time(np_time)
    return f'{long_name}\nTime: {py_time:%Y-%m-%d %H:%M}'

# Constrain our dataset to just a subset of the thousands of available timesteps
# (In this case, the most recent 20) And also a single depth (we are using k=-3)
time_anim = dataset.isel(time=slice(-20, None), k=-3).ems.animate_on_figure(
    figure=Figure(figsize=(8, 5), dpi=100),
    scalar='Chl_a_sum', 
    coordinate='time',
    title=make_title,
    interval=100
)

# Render our animation to this notebook
HTML(time_anim.to_jshtml())


### Animate a horizontal layer over depth

Because eReefs model results are multidimensional, we can also animate across axes *other* than time.

Use emsarray to create an animation of how the Total Chlorophyll (Chl_a_sum) variable from the model results changes with depth for the last time in the dataset:

In [ ]:
depth_anim = dataset.isel(time=-1).ems.animate_on_figure(
    figure=Figure(figsize=(8, 5), dpi=100),
    scalar='Chl_a_sum', 
    coordinate='k',
    interval=100, 
    repeat='bounce'
)

# Render the depth animation to this notebook
HTML(depth_anim.to_jshtml())


### Animations in video format

Animations can be rendered to video files in two different ways. Both approaches require ffmpeg to be installed. If you do not already have this installed, you can install it with:

`conda install ffmpeg`

#### Render your animation to embedded HTML5 video

Working with the same depth-animation we created above:


In [ ]:
HTML(depth_anim.to_html5_video())

#### Render your animation to mp4

Useful to export your animation to a file for offline use

In [ ]:
import os.path
animation_path = os.path.abspath('_static/gbr4_chl_depth_animation.mp4')
depth_anim.save(animation_path, writer='ffmpeg_file')
print("Saved to", animation_path)

And use that file as needed:

In [ ]:
import IPython.display
IPython.display.Video(animation_path, embed=True)